# Imports

In [ ]:
import pandas as pd
import numpy as np

from sklearn.svm import SVC 
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import SMOTE
from imblearn.ensemble import EasyEnsembleClassifier
from imblearn.pipeline import make_pipeline
from imblearn.metrics import classification_report_imbalanced

# Check the data

In [ ]:
data_df = pd.read_csv("/content/carclaims.csv")
data_df

,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,...,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange-Claim,NumberOfCars,Year,BasePolicy,FraudFound
0,Dec,5,Wednesday,Honda,Urban,Tuesday,Jan,1,Female,Single,...,26 to 30,No,No,External,none,1 year,3 to 4,1994,Liability,No
1,Jan,3,Wednesday,Honda,Urban,Monday,Jan,4,Male,Single,...,31 to 35,Yes,No,External,none,no change,1 vehicle,1994,Collision,No
2,Oct,5,Friday,Honda,Urban,Thursday,Nov,2,Male,Married,...,41 to 50,No,No,External,none,no change,1 vehicle,1994,Collision,No
3,Jun,2,Saturday,Toyota,Rural,Friday,Jul,1,Male,Married,...,51 to 65,Yes,No,External,more than 5,no change,1 vehicle,1994,Liability,No
4,Jan,5,Monday,Honda,Urban,Tuesday,Feb,2,Female,Single,...,31 to 35,No,No,External,none,no change,1 vehicle,1994,Collision,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15415,Nov,4,Friday,Toyota,Urban,Tuesday,Nov,5,Male,Married,...,31 to 35,No,No,External,none,no change,1 vehicle,1996,Collision,Yes
15416,Nov,5,Thursday,Pontiac,Urban,Friday,Dec,1,Male,Married,...,31 to 35,No,No,External,more than 5,no change,3 to 4,1996,Liability,No
15417,Nov,5,Thursday,Toyota,Rural,Friday,Dec,1,Male,Single,...,26 to 30,No,No,External,1 to 2,no change,1 vehicle,1996,Collision,Yes
15418,Dec,1,Monday,Toyota,Urban,Thursday,Dec,2,Female,Married,...,31 to 35,No,No,External,more than 5,no change,1 vehicle,1996,All Perils,No


In [ ]:
data_df.isna().sum()

Month                   0
WeekOfMonth             0
DayOfWeek               0
Make                    0
AccidentArea            0
DayOfWeekClaimed        0
MonthClaimed            0
WeekOfMonthClaimed      0
Sex                     0
MaritalStatus           0
Age                     0
Fault                   0
PolicyType              0
VehicleCategory         0
VehiclePrice            0
PolicyNumber            0
RepNumber               0
Deductible              0
DriverRating            0
Days:Policy-Accident    0
Days:Policy-Claim       0
PastNumberOfClaims      0
AgeOfVehicle            0
AgeOfPolicyHolder       0
PoliceReportFiled       0
WitnessPresent          0
AgentType               0
NumberOfSuppliments     0
AddressChange-Claim     0
NumberOfCars            0
Year                    0
BasePolicy              0
FraudFound              0
dtype: int64

In [ ]:
data_df.columns

Index(['Month', 'WeekOfMonth', 'DayOfWeek', 'Make', 'AccidentArea',
       'DayOfWeekClaimed', 'MonthClaimed', 'WeekOfMonthClaimed', 'Sex',
       'MaritalStatus', 'Age', 'Fault', 'PolicyType', 'VehicleCategory',
       'VehiclePrice', 'PolicyNumber', 'RepNumber', 'Deductible',
       'DriverRating', 'Days:Policy-Accident', 'Days:Policy-Claim',
       'PastNumberOfClaims', 'AgeOfVehicle', 'AgeOfPolicyHolder',
       'PoliceReportFiled', 'WitnessPresent', 'AgentType',
       'NumberOfSuppliments', 'AddressChange-Claim', 'NumberOfCars', 'Year',
       'BasePolicy', 'FraudFound'],
      dtype='object')

In [ ]:
for col in data_df.columns:
  print(col)
  print(data_df[col].value_counts())
  print()

Month
Jan    1411
May    1367
Mar    1360
Jun    1321
Oct    1305
Dec    1285
Apr    1280
Feb    1266
Jul    1257
Sep    1240
Nov    1201
Aug    1127
Name: Month, dtype: int64

WeekOfMonth
3    3640
2    3558
4    3398
1    3187
5    1637
Name: WeekOfMonth, dtype: int64

DayOfWeek
Monday       2616
Friday       2445
Tuesday      2300
Thursday     2173
Wednesday    2159
Saturday     1982
Sunday       1745
Name: DayOfWeek, dtype: int64

Make
Pontiac      3837
Toyota       3121
Honda        2801
Mazda        2354
Chevrolet    1681
Accura        472
Ford          450
VW            283
Dodge         109
Saab          108
Mercury        83
Saturn         58
Nisson         30
BMW            15
Jaguar          6
Porche          5
Mecedes         4
Ferrari         2
Lexus           1
Name: Make, dtype: int64

AccidentArea
Urban    13822
Rural     1598
Name: AccidentArea, dtype: int64

DayOfWeekClaimed
Monday       3757
Tuesday      3375
Wednesday    2951
Thursday     2660
Friday       2497
Satu

# Read and Preprocess the data

In [ ]:
# Convert categorical values to numeric
cols_to_keep_as_they_are = ['WeekOfMonth', 'WeekOfMonthClaimed', 'Age', 'PolicyNumber', 'RepNumber', 'DriverRating']
for col in data_df.drop(cols_to_keep_as_they_are, axis=1).columns:
  unique_vals = list(data_df[col].unique())
  replaced_vals = list(np.arange(len(unique_vals)))
  data_df[col].replace(unique_vals, replaced_vals, inplace=True)

data_df.head()

,Month,WeekOfMonth,DayOfWeek,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,...,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange-Claim,NumberOfCars,Year,BasePolicy,FraudFound
0,0,5,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,3,0,0,0,1,0,4,1,0,...,1,1,0,0,0,1,1,0,1,0
2,2,5,1,0,0,2,1,2,1,1,...,2,0,0,0,0,1,1,0,1,0
3,3,2,2,1,1,3,2,1,1,1,...,3,1,0,0,1,1,1,0,0,0
4,1,5,3,0,0,0,3,2,0,0,...,1,0,0,0,0,1,1,0,1,0


In [ ]:
# abs(data_df.corr()['FraudFound']).sort_values(ascending=False)

In [ ]:
# PARAMETERS
TARGET = 'FraudFound'
TEST_SIZE = 0.25
RANDOM_STATE = 0

# Split to train and test sets 
X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(data_df.drop(TARGET, axis=1),
                                                                data_df[TARGET], 
                                                                test_size=TEST_SIZE,
                                                                shuffle=True,
                                                                random_state=RANDOM_STATE)

# Scale data
scaler = MinMaxScaler()
X_train_sc = scaler.fit_transform(X_train_df)
X_train_df_sc = pd.DataFrame(X_train_sc, columns=X_train_df.columns, index=X_train_df.index)
X_test_sc = scaler.transform(X_test_df)
X_test_df_sc = pd.DataFrame(X_test_sc, columns=X_test_df.columns, index=X_test_df.index)

print(X_train_df.shape, X_test_df.shape)
print(y_train_df.shape, y_test_df.shape)

# Change 0's to -1's, because SVM requires that labels are +1 and -1
y_train_df[y_train_df == 0.] = -1
y_test_df[y_test_df == 0.] = -1

(11565, 32) (3855, 32)
(11565,) (3855,)


# 1. Synthetic Oversampling

In [ ]:
# create the classifiers
classifiers = {
    'random forest': RandomForestClassifier(n_estimators=100, random_state=0),
    'linear SVM': SVC(kernel='linear'),
    'naive bayes': GaussianNB(),
}

y_test = y_test_df.to_numpy(dtype='int64')

for name in classifiers.keys():
  print(f'======================== {name} ========================')

  print("\n***** WITHOUT over-sampling *****")
  pipeline = make_pipeline(classifiers[name])
  pipeline.fit(X_train_df, y_train_df)
  y_preds = pipeline.predict(X_test_df)
  print(classification_report_imbalanced(y_test, y_preds))

  print("\n***** WITH over-sampling*****")
  pipeline = make_pipeline(SMOTE(random_state=0, k_neighbors=5),
                           classifiers[name])
  pipeline.fit(X_train_df, y_train_df)
  y_preds = pipeline.predict(X_test_df)
  print(classification_report_imbalanced(y_test, y_preds))

======================== random forest ========================

***** WITHOUT over-sampling *****
                   pre       rec       spe        f1       geo       iba       sup

         -1       0.94      1.00      0.02      0.97      0.15      0.02      3624
          1       0.71      0.02      1.00      0.04      0.15      0.02       231

avg / total       0.93      0.94      0.08      0.91      0.15      0.02      3855


***** WITH over-sampling*****
                   pre       rec       spe        f1       geo       iba       sup

         -1       0.95      0.97      0.12      0.96      0.34      0.13      3624
          1       0.22      0.12      0.97      0.16      0.34      0.11       231

avg / total       0.90      0.92      0.17      0.91      0.34      0.13      3855

======================== linear SVM ========================

***** WITHOUT over-sampling *****
                   pre       rec       spe        f1       geo       iba       sup

         -1       0.

Από τα παραπάνω αποτελέσματα παρατηρώ τα παρακάτω:
1. Για το **Random Forest**:
  * όταν εφαρμόζεται *synthetic oversampling* μειώνονται *ελάχιστα* οι μετρικές *precision* και *recall*, ωστόσο όλες οι υπόλοιπες μετρικές, εκτός της *f1*, που *παραμένει ίδια*, βελτιώνοντα σημαντικά.

2. Για το **Linear SVM**, όταν εφαρμόζεται *synthetic oversampling*:
  * μειώνεται *ελάχιστα* η μετρική *precision* 
  * μειώνονται *σημαντικά* οι μετρικές *recall* και *f1*
  * ωστόσο όλες οι υπόλοιπες μετρικές βελτιώνοντα σημαντικά.

3. Για το **Naive Bayes**, όταν εφαρμόζεται *synthetic oversampling*::
  * βελτιώνεται *ελάχιστα* η μετρική *precision* 
  * μειώνονται *σημαντικά* οι μετρικές *recall* και *f1*
  * ωστόσο όλες οι υπόλοιπες μετρικές βελτιώνοντα σημαντικά.

Συνολικά, παρατηρούμε ότι:
* στην προκειμένη περίπτωση **με τη χρήση του *synthetic oversampling* επιτυγχάνεται ισορροπία στην επίδοση όλων των μετρικών και στους 3 παραπάνω αλγορίθμους**
* θα μπορούσαμε να πούμε ότι **η χρήση του *synthetic oversampling* ΔΕΝ συνίσταται όταν όταν μας ενδιαφέρει να βελτιστοποιήσουμε κάποια από τις μετρικές *precision*, *recall* ή *f1***.


# 2. Undersampling - NearMiss

In [ ]:
# create the classifiers
classifiers = {
    'random forest': RandomForestClassifier(n_estimators=100, random_state=0),
    'linear SVM': SVC(kernel='linear'),
    'naive bayes': GaussianNB(),
}

y_test = y_test_df.to_numpy(dtype='int64')

for name in classifiers.keys():
  print(f'======================== {name} ========================')

  print("\n***** WITHOUT under-sampling *****")
  pipeline = make_pipeline(classifiers[name])
  pipeline.fit(X_train_df, y_train_df)
  y_preds = pipeline.predict(X_test_df)
  print(classification_report_imbalanced(y_test, y_preds))

  print("\n***** WITH NearMiss - version 1 *****")
  pipeline = make_pipeline(NearMiss(version=1),
                           classifiers[name])
  pipeline.fit(X_train_df, y_train_df)
  y_preds = pipeline.predict(X_test_df)
  print(classification_report_imbalanced(y_test, y_preds))

  print("\n***** WITH NearMiss - version 2 *****")
  pipeline = make_pipeline(NearMiss(version=2),
                           classifiers[name])
  pipeline.fit(X_train_df, y_train_df)
  y_preds = pipeline.predict(X_test_df)
  print(classification_report_imbalanced(y_test, y_preds))

  print("\n***** WITH NearMiss - version 3 *****")
  pipeline = make_pipeline(NearMiss(version=3),
                           classifiers[name])
  pipeline.fit(X_train_df, y_train_df)
  y_preds = pipeline.predict(X_test_df)
  print(classification_report_imbalanced(y_test, y_preds))

======================== random forest ========================

***** WITHOUT under-sampling *****
                   pre       rec       spe        f1       geo       iba       sup

         -1       0.94      1.00      0.02      0.97      0.15      0.02      3624
          1       0.71      0.02      1.00      0.04      0.15      0.02       231

avg / total       0.93      0.94      0.08      0.91      0.15      0.02      3855


***** WITH NearMiss - version 1 *****
                   pre       rec       spe        f1       geo       iba       sup

         -1       0.99      0.63      0.90      0.77      0.75      0.55      3624
          1       0.13      0.90      0.63      0.23      0.75      0.58       231

avg / total       0.94      0.65      0.88      0.74      0.75      0.55      3855


***** WITH NearMiss - version 2 *****
                   pre       rec       spe        f1       geo       iba       sup

         -1       0.97      0.11      0.94      0.20      0.33      

Από τα παραπάνω αποτελέσματα παρατηρώ τα παρακάτω:
1. Για το **Random Forest**:
  * όταν εφαρμόζεται η τεχνική *NearMiss 1* μειώνονται *σημαντικά* οι μετρικές *recall* και *f1*, ωστόσο όλες οι υπόλοιπες μετρικές βελτιώνοντα σημαντικά.
  * όταν εφαρμόζεται η τεχνική *NearMiss 2* μειώνονται *σημαντικά* οι μετρικές *recall* και *f1*, ωστόσο όλες οι υπόλοιπες μετρικές βελτιώνοντα σημαντικά, εκτός της *precision* που μειώνεται ελάχιστα.
  * όταν εφαρμόζεται η τεχνική *NearMiss 3* ισχύουν τα ίδια με την *NearMiss 1*.

2. Για το **Linear SVM**, όταν εφαρμόζεται *synthetic oversampling*:
  * όταν εφαρμόζεται η τεχνική *NearMiss 1* μειώνονται *σημαντικά* οι μετρικές *recall* και *f1*, ωστόσο όλες οι υπόλοιπες μετρικές βελτιώνοντα σημαντικά.
  * όταν εφαρμόζονται οι τεχνικές *NearMiss 2* ή *NearMiss 3* ισχύουν τα ίδια με την *NearMiss 1*.

3. Για το **Naive Bayes**, όταν εφαρμόζεται *synthetic oversampling*::
  * όταν εφαρμόζεται η τεχνική *NearMiss 1* μειώνονται *σημαντικά* οι μετρικές *recall* και *f1*, ωστόσο όλες οι υπόλοιπες μετρικές βελτιώνοντα σημαντικά.
  * όταν εφαρμόζεται η τεχνική *NearMiss 2* μειώνονται *ακόμη περισ΄΄οτερο* οι μετρικές *recall* και *f1*, μειώνονται η *geo* και η *iba*, ωστόσο όλες οι υπόλοιπες μετρικές βελτιώνοντα σημαντικά.
  * όταν εφαρμόζεται η τεχνική *NearMiss 3* ισχύουν τα ίδια με την *NearMiss 1*.

Συνολικά, παρατηρούμε ότι:
* στην προκειμένη περίπτωση **με τη χρήση του *NearMiss* επιτυγχάνεται ισορροπία στην επίδοση όλων των μετρικών και στους 3 παραπάνω αλγορίθμους**.
* θα μπορούσαμε να πούμε ότι **η χρήση του *NearMiss* ΔΕΝ συνίσταται όταν όταν μας ενδιαφέρει να βελτιστοποιήσουμε κάποια από τις μετρικές *precision*, *recall* ή *f1***.
* Θα πρέπει να ελέγχεται ποιός από τους αλγορίθμους *NearMiss 1* ή *NearMiss 2* είναι καλύτερο στην κάθε περίπτωση.
* **Όπως είχαμε πει στο μάθημα, ο *NearMiss 3* δεν φαίνεται να αποτελεί ξεχωριστό αλγόριθμο, καθώς και στις τρεις περιπτώσεις πετυχαίνει τα ίδια αποτελέσματα σε σχέση με το *NearMiss 1***.

# 3. EasyEnsemble

In [ ]:
# create the classifiers
classifiers = {
    'random forest': RandomForestClassifier(n_estimators=100, random_state=0),
    'linear SVM': SVC(kernel='linear'),
    'naive bayes': GaussianNB(),
}

y_test = y_test_df.to_numpy(dtype='int64')

for name in classifiers.keys():
  print(f'======================== {name} ========================')

  print("\n***** WITHOUT EasyEnsemble *****")
  pipeline = make_pipeline(classifiers[name])
  pipeline.fit(X_train_df, y_train_df)
  y_preds = pipeline.predict(X_test_df)
  print(classification_report_imbalanced(y_test, y_preds))

  print("\n***** WITH EasyEnsemble *****")
  pipeline = make_pipeline(EasyEnsembleClassifier(random_state=42))
  pipeline.fit(X_train_df, y_train_df)
  y_preds = pipeline.predict(X_test_df)
  print(classification_report_imbalanced(y_test, y_preds))


======================== random forest ========================

***** WITHOUT EasyEnsemble *****
                   pre       rec       spe        f1       geo       iba       sup

         -1       0.94      1.00      0.02      0.97      0.15      0.02      3624
          1       0.71      0.02      1.00      0.04      0.15      0.02       231

avg / total       0.93      0.94      0.08      0.91      0.15      0.02      3855


***** WITH EasyEnsemble *****
                   pre       rec       spe        f1       geo       iba       sup

         -1       0.99      0.62      0.86      0.76      0.73      0.52      3624
          1       0.13      0.86      0.62      0.22      0.73      0.54       231

avg / total       0.93      0.63      0.84      0.73      0.73      0.52      3855

======================== linear SVM ========================

***** WITHOUT EasyEnsemble *****
                   pre       rec       spe        f1       geo       iba       sup

         -1       0.94

Από τα παραπάνω αποτελέσματα παρατηρώ τα παρακάτω:
1. Για το **Random Forest**:
  * όταν εφαρμόζεται o *Easy Ensemble* μειώνονται *σημαντικά* οι μετρικές *f1* και *recall*, ωστόσο όλες οι υπόλοιπες μετρικές, εκτός της *precision*, που *παραμένει ίδια*, βελτιώνοντα σημαντικά.

2. Για το **Linear SVM**, όταν εφαρμόζεται *synthetic oversampling*:
  * όταν εφαρμόζεται o *Easy Ensemble* μειώνονται *σημαντικά* οι μετρικές *f1* και *recall*, ωστόσο όλες οι υπόλοιπες μετρικές, εκτός της *precision*, που *βελτιώνεται ελάχιστα*, βελτιώνοντα σημαντικά.

3. Για το **Naive Bayes**, όταν εφαρμόζεται *synthetic oversampling*::
  * όταν εφαρμόζεται o *Easy Ensemble* μειώνονται *σημαντικά* οι μετρικές *f1* και *recall*, ωστόσο όλες οι υπόλοιπες μετρικές, εκτός της *precision*, που *βελτιώνεται ελάχιστα*, βελτιώνοντα σημαντικά.

Συνολικά, παρατηρούμε ότι:
* στην προκειμένη περίπτωση **με τη χρήση του *Easy Ensemble* επιτυγχάνεται ισορροπία στην επίδοση όλων των μετρικών και στους 3 παραπάνω αλγορίθμους**
* θα μπορούσαμε να πούμε ότι **η χρήση του *synthetic oversampling* ΔΕΝ συνίσταται όταν όταν μας ενδιαφέρει να βελτιστοποιήσουμε κάποια από τις μετρικές *recall* ή *f1***.
